In [1]:
from pynq import DefaultIP
from pynq import Overlay
from pynq import Xlnk
import pynq.lib.dma
import numpy as np

In [2]:
class ConstantMultiplyDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)

    bindto = ['xilinx.com:hls:mult_constant:1.0']

    @property
    def constant(self):
        return self.read(0x10)

    @constant.setter
    def constant(self, value):
        self.write(0x10, value)

In [3]:
overlay = Overlay('axis_mult.bit')
overlay?

In [4]:
dma = overlay.mult.axi_dma
multiply = overlay.mult.axi_mult

In [5]:
xlnk = Xlnk()
in_buffer = xlnk.cma_array(shape=(5,), dtype=np.uint32)
out_buffer = xlnk.cma_array(shape=(5,), dtype=np.uint32)

for i in range(5):
    in_buffer[i] = i

multiply.constant = 10
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()

out_buffer

ContiguousArray([ 0, 10, 20, 30, 40], dtype=uint32)